PURPOSE:
------------------------------
This creates a list of icustayIDs of patients who develop sepsis at some point 
in the ICU. records charttime for onset of sepsis. Uses sepsis3 criteria

STEPS:
-------------------------------
IMPORT DATA FROM CSV FILES  
FLAG PRESUMED INFECTION  
PREPROCESSING  
REFORMAT in 4h time slots  
COMPUTE SOFA at each time step  
FLAG SEPSIS  

note: the process generates the same features as the final MDP dataset, most of which are not used to compute SOFA

# Import all data

In [1]:
import pandas as pd
import numpy as np
exportdir='/data/localhost/taufiq/export-dir/'

In [ ]:
import logging
logging.basicConfig(filename='sepsisDef-1hourly.log', level=logging.INFO)

In [2]:
abx = pd.read_csv(exportdir + '/abx.csv', sep='|')
culture = pd.read_csv(exportdir + '/culture.csv', sep='|')
microbio = pd.read_csv(exportdir + '/microbio.csv', sep='|')
demog = pd.read_csv(exportdir + '/demog.csv', sep='|')
ce010 = pd.read_csv(exportdir + '/ce010000.csv', sep='|')
ce1020 = pd.read_csv(exportdir + '/ce1000020000.csv', sep='|')
ce2030 = pd.read_csv(exportdir + '/ce2000030000.csv', sep='|')
ce3040 = pd.read_csv(exportdir + '/ce3000040000.csv', sep='|')
ce4050 = pd.read_csv(exportdir + '/ce4000050000.csv', sep='|')
ce5060 = pd.read_csv(exportdir + '/ce5000060000.csv', sep='|')
ce6070 = pd.read_csv(exportdir + '/ce6000070000.csv', sep='|')
ce7080 = pd.read_csv(exportdir + '/ce7000080000.csv', sep='|')
ce8090 = pd.read_csv(exportdir + '/ce8000090000.csv', sep='|')
ce90100 = pd.read_csv(exportdir + '/ce90000100000.csv', sep='|')
lab_ce = pd.read_csv(exportdir + '/labs_ce.csv', sep='|').rename(columns = {'charttime': 'timestp'}, inplace = False)
lab_le = pd.read_csv(exportdir + '/labs_le.csv', sep='|')
labU = pd.concat([lab_ce, lab_le], ignore_index = True)
MV = pd.read_csv(exportdir + '/mechvent.csv', sep='|')
inputpreadm = pd.read_csv(exportdir + '/preadm_fluid.csv', sep='|')
inputMV = pd.read_csv(exportdir + '/fluid_mv.csv', sep='|')
inputCV = pd.read_csv(exportdir + '/fluid_cv.csv', sep='|')
vasoMV = pd.read_csv(exportdir + '/vaso_mv.csv', sep='|')
vasoCV = pd.read_csv(exportdir + '/vaso_cv.csv', sep='|')
UOpreadm = pd.read_csv(exportdir + '/preadm_uo.csv', sep='|')
UO = pd.read_csv(exportdir + '/uo.csv', sep='|')

reflabs = pd.read_csv('./Reflabs.csv', header=None)
refvitals = pd.read_csv('./Refvitals.csv', header=None)
sample_and_hold = pd.read_csv('./sample_and_hold.csv')

# Initial Data Manipulations

In [3]:
# Change the headers of sample_and_hold to remove ''' '''
for i in sample_and_hold:
    sample_and_hold.rename(columns={i:i.replace("'","")}, inplace=True)
    
sample_and_hold.head()

,Height_cm,Weight_kg,GCS,RASS,HR,SysBP,MeanBP,DiaBP,RR,SpO2,...,Arterial_pH,paO2,paCO2,Arterial_BE,Arterial_lactate,HCO3,ETCO2,SvO2,mechvent,extubated
0,168,72,6,6,2,2,2,2,2,2,...,8,8,8,8,8,8,8,8,6,6


In [4]:
# if charttime is empty but chartdate isn't
microbio.loc[microbio['charttime'].isnull(), 'charttime'] = microbio['chartdate'] 
microbio = microbio.drop(columns = 'chartdate')
bacterio = pd.concat([microbio, culture], ignore_index = True, sort=True)

In [5]:
demog['morta_90'] = demog['morta_90'].fillna(0)
demog['morta_hosp'] = demog['morta_hosp'].fillna(0)
demog['elixhauser'] = demog['elixhauser'].fillna(0)

In [6]:
inputMV.head()
inputMV['normrate'] = inputMV['rate']*inputMV['tev']/inputMV['amount']

In [7]:
def find_icustay_id_from_demog(hadm_id, time, subject_id = None):
    if subject_id is not None:
        df = demog.loc[demog['subject_id'] == subject_id]
    else:
        df = demog.loc[demog['hadm_id'] == hadm_id]
    for index, row in df.iterrows():
        if (time >= row['intime'] - 48*3600 and time <= row['outtime'] + 48*3600) or len(df) == 1:
            return row['icustay_id']
    df2 = df.loc[df['hadm_id'] == hadm_id]
    if len(df2) == 1:
        return df2['icustay_id'].values[0]
    return None

In [8]:
bacterio.head()
# Fill in missing ICUSTAY IDs in bacterio
for index, row in bacterio.iterrows():
    if pd.isna(row['icustay_id']):
        charttime, hadm_id, subject_id = row['charttime'], row['hadm_id'], row['subject_id']
        icustayid = find_icustay_id_from_demog(hadm_id, charttime, subject_id)
        if icustayid is not None:
            bacterio.at[index,'icustay_id'] = icustayid

In [9]:
# Fill in missing ICUSTAY IDs in abx
for index, row in abx.iterrows():
    if pd.isna(row['icustay_id']):
        time, hadm_id = row['startdate'], row['hadm_id']
        icustayid = find_icustay_id_from_demog(hadm_id, time)
        if icustayid is not None:
            abx.at[index,'icustay_id'] = icustayid

# Find presumed onset of infection according to sepsis3 guidelines

In [10]:
from sklearn.metrics.pairwise import nan_euclidean_distances

onset = pd.DataFrame(columns=['subject_id', 'icustay_id', 'onsettime'])

for icustayid in range(1,100001):
    ab = pd.Series.to_numpy(abx.loc[abx['icustay_id'] == icustayid + 200000, 'startdate'])
    bact = pd.Series.to_numpy(bacterio.loc[bacterio['icustay_id'] == icustayid + 200000, 'charttime'])
    subj_bact = pd.Series.to_numpy(bacterio.loc[bacterio['icustay_id'] == icustayid + 200000, 'subject_id'])
    
    if len(ab) > 0 and len(bact) > 0:
        D = nan_euclidean_distances(ab.reshape(-1,1), bact.reshape(-1,1))/3600
        for i in range(len(D)):
            M = min(D[i])
            I = D[i].argmin()
            ab1 = ab[i]
            bact1 = bact[I]
            
            if M <= 24 and ab1 <= bact1:
                onset = onset.append({'subject_id': subj_bact[0], 'icustay_id': icustayid, 'onsettime': ab1}, ignore_index=True)
                break
            elif M <= 72 and ab1 >= bact1:
                onset = onset.append({'subject_id': subj_bact[0], 'icustay_id': icustayid, 'onsettime': bact1}, ignore_index=True)
                break 
        

In [11]:
len(onset.loc[onset['onsettime']>0])

26332

# Replacing item_ids with column numbers from reference tables

In [12]:
import os

def find_the_col_number(df, num):
    for index, row in df.iterrows():
        arr = pd.Series.to_numpy(row)
        if num in arr:
            return int(index+1)
        
def replace_itemids_with_col_num(df, reftable):
    logging.info("Started replacing itemids")
    for index, row in df.iterrows():
        df.at[index, 'colnum'] = find_the_col_number(reftable, row['itemid'])
        if index % 1000000 == 0:
            logging.info('On index: ' + str(index))
    df.drop(columns=['itemid'], inplace=True)
    df['colnum'] = df['colnum'].astype({'colnum': 'int64'})

In [13]:
replace_itemids_with_col_num(labU, reflabs)
replace_itemids_with_col_num(ce010,refvitals)
replace_itemids_with_col_num(ce1020,refvitals)
replace_itemids_with_col_num(ce2030,refvitals)
replace_itemids_with_col_num(ce3040,refvitals)
replace_itemids_with_col_num(ce4050,refvitals)
replace_itemids_with_col_num(ce5060,refvitals)
replace_itemids_with_col_num(ce6070,refvitals)
replace_itemids_with_col_num(ce7080,refvitals)
replace_itemids_with_col_num(ce8090,refvitals)
replace_itemids_with_col_num(ce90100,refvitals)

# Initial reformat with chartevents, labs and mechvent

gives an array with all unique charttime (1 per row) and all items in columns.
## IMPORTANT 
Here i use -48 -> +24 because that's for sepsis3 cohort defintion!!  
I need different time period for the MDP (-24 -> +48)

In [14]:
winb4=49   #lower limit for inclusion of data (48h before time flag)
winaft=25  # upper limit (24h after)
irow = 0
reformat, qstime = pd.DataFrame(), pd.DataFrame()

logging.info('Initial reformat with chartevents, labs and mechvent')
for icustayid in range(1,100001):
    if icustayid % 10000 == 0:
        logging.info('On icustayid: ' + str(icustayid) + '/100000')
    qst = onset.loc[onset['icustay_id'] == icustayid, 'onsettime']
    if len(qst) > 0 and qst.values[0] > 0:
        d1 = demog.loc[demog['icustay_id'] == icustayid + 200000, ['age', 'dischtime']]
        
        if len(d1['age']) > 0 and d1['age'].values[0] > 6574:
            if icustayid < 10000:
                temp = ce010.loc[ce010['icustay_id'] == icustayid + 200000]
            elif icustayid < 20000:
                temp = ce1020.loc[ce1020['icustay_id'] == icustayid + 200000]
            elif icustayid < 30000:
                temp = ce2030.loc[ce2030['icustay_id'] == icustayid + 200000]
            elif icustayid < 40000:
                temp = ce3040.loc[ce3040['icustay_id'] == icustayid + 200000]
            elif icustayid < 50000:
                temp = ce4050.loc[ce4050['icustay_id'] == icustayid + 200000]
            elif icustayid < 60000:
                temp = ce5060.loc[ce5060['icustay_id'] == icustayid + 200000]
            elif icustayid < 70000:
                temp = ce6070.loc[ce6070['icustay_id'] == icustayid + 200000]
            elif icustayid < 80000:
                temp = ce7080.loc[ce7080['icustay_id'] == icustayid + 200000]
            elif icustayid < 90000:
                temp = ce8090.loc[ce8090['icustay_id'] == icustayid + 200000]
            else:
                temp = ce90100.loc[ce90100['icustay_id'] == icustayid + 200000]
            
            temp = temp.loc[temp['charttime'].between(qst.values[0]-(winb4+4)*3600,qst.values[0]+(winaft+4)*3600)] #time period of interest
            
            # LABEVENTS
            temp2 = labU.loc[labU['icustay_id'] == icustayid + 200000]
            temp2 = temp2.loc[temp2['timestp'].between(qst.values[0]-(winb4+4)*3600,qst.values[0]+(winaft+4)*3600)] #time period of interest
            
            #MECH VENT and Exubated
            temp3 = MV.loc[MV['icustay_id'] == icustayid + 200000]
            temp3 = temp3.loc[temp3['charttime'].between(qst.values[0]-(winb4+4)*3600, qst.values[0]+(winaft+4)*3600)] #time period of interest
            
            unique_timestp = pd.concat([temp['charttime'], temp2['timestp'].rename('charttime'), temp3['charttime']]).unique()
            unique_timestp.sort() # list of unique timestamps from all 3 sources sorted in ascending order
            
            for i in range(len(unique_timestp)):
                timedata = temp.loc[temp['charttime'] == unique_timestp[i]]
                
                reformat.at[irow, 'timestep'] = i
                reformat.at[irow, 'icustay_id'] = icustayid
                reformat.at[irow, 'charttime'] = unique_timestp[i]
                
                for index, row in timedata.iterrows():
                    col = row['colnum']
                    value = row['valuenum']
                    reformat.at[irow, 3 + col] = value

                # LAB values
                timedata = temp2.loc[temp2['timestp'] == unique_timestp[i]]
                for index, row in timedata.iterrows():
                    col = row['colnum']
                    value = row['valuenum']
                    reformat.at[irow, 31+col] = value;

                # MV
                value = temp3.loc[temp3['charttime'] == unique_timestp[i], ['mechvent', 'extubated']]
                if len(value) > 0:
                    reformat.at[irow, 'mechvent'] = value['mechvent'].values[0]
                    reformat.at[irow, 'extubated'] = value['extubated'].values[0]
                
                irow = irow + 1
            
            if len(unique_timestp) > 0:
                qstime = qstime.append({'onset': qst.values[0], 'icustay_id': icustayid, 'firsttimestp': unique_timestp[0], 'lasttimestp':unique_timestp[-1], 'dischtime': d1['dischtime'].values[0]}, ignore_index = True)

reformat['timestep'] = reformat['timestep'].astype({'timestep': 'int64'})
reformat['icustay_id'] = reformat['icustay_id'].astype({'icustay_id': 'int64'})
            

Map the column numbers in reformat to the corresponding column headers

In [15]:
sah_headers = np.array(sample_and_hold.columns)

for i in range(4,69):
    if i in reformat:
        reformat.rename(columns={i: sah_headers[i-4]}, inplace=True)
    else:
        reformat[sah_headers[i-4]] = np.nan

# Outliers

In [16]:
def deloutbelow(df, var, thres):
    for index, row in df.iterrows():
        if row[var] < thres:
            df.at[index, var] = np.nan
            
def deloutabove(df, var, thres):
    for index, row in df.iterrows():
        if row[var] > thres:
            df.at[index, var] = np.nan

In [17]:
# Weight
deloutabove(reformat,"Weight_kg",300)
# HR
deloutabove(reformat,"HR",250)
# BP
deloutabove(reformat,"SysBP",300)
deloutbelow(reformat,"MeanBP",0)
deloutabove(reformat,"MeanBP",200)
deloutbelow(reformat,"DiaBP",0)
deloutabove(reformat,"DiaBP",200)
# RR
deloutabove(reformat,"RR",80)
# Sp02
deloutabove(reformat,"SpO2",150)
deloutbelow(reformat,"SpO2",50)
if "SpO2" in reformat:
    reformat.loc[reformat["SpO2"] > 100, "SpO2"] = 100
# temp
if "Temp_C" in reformat and "Temp_F" in reformat:
    reformat.loc[(reformat["Temp_C"] > 90) & (reformat["Temp_F"].isnull()), "Temp_F"] = reformat["Temp_C"]
deloutabove(reformat,"Temp_C",90)
deloutbelow(reformat,"Temp_C",25)
# Fi02
deloutabove(reformat,"FiO2_100",100)
if "FiO2_100" in reformat:
    reformat.loc[reformat["FiO2_100"] < 1, "FiO2_100"] = reformat["FiO2_100"]*100
deloutbelow(reformat,"FiO2_100",20)
deloutabove(reformat,"FiO2_1",1.5)
# O2 Flow
deloutabove(reformat,"O2flow",70)
# PEEP
deloutbelow(reformat,"PEEP",0)
deloutabove(reformat,"PEEP",40)
# TV
deloutabove(reformat,"TidalVolume",1800)
# MV
deloutabove(reformat,"MinuteVentil",50)
# K+
deloutbelow(reformat,"Potassium",1)
deloutabove(reformat,"Potassium",15)
# Na
deloutbelow(reformat,"Sodium",95)
deloutabove(reformat,"Sodium",178)
# Cl
deloutbelow(reformat,"Chloride",70)
deloutabove(reformat,"Chloride",150)
# Glc
deloutbelow(reformat,"Glucose",1)
deloutabove(reformat,"Glucose",1000)
# Creat
deloutabove(reformat,"Creatinine",150)
# Mg
deloutabove(reformat,"Magnesium",10)
# Ca
deloutabove(reformat,"Calcium",20)
# ionized Ca
deloutabove(reformat,"Ionised_Ca",5)
# CO2
deloutabove(reformat,"CO2_mEqL",120)
# SGPT/SGOT
deloutabove(reformat,"SGOT",10000)
deloutabove(reformat,"SGPT",10000)
# Hb/Ht
deloutabove(reformat,"Hb",20)
deloutabove(reformat,"Ht",65)
# WBC
deloutabove(reformat,"WBC_count",500)
# plt
deloutabove(reformat,"Platelets_count",2000)
# INR
deloutabove(reformat,"INR",20)
# pH
deloutbelow(reformat,"Arterial_pH",6.7)
deloutabove(reformat,"Arterial_pH",8)
# po2
deloutabove(reformat,"paO2",700)
# pco2
deloutabove(reformat,"paCO2",200)
# BE
deloutbelow(reformat,"Arterial_BE",-50)
# lactate
deloutabove(reformat,"Arterial_lactate",30)

Some more manipulation/imputation from existing values

In [18]:
# Sample and hold function
def SAH(temp, vitalslab_hold):
    temp_copy = temp.copy()
    oldstayid = temp_copy['icustay_id'].values[0]
    lastcharttime = {}
    lastvalue = {}
    
    for col in temp_copy:
        if col in vitalslab_hold:

            for index, row in temp_copy.iterrows():
                if oldstayid != row['icustay_id']:
                    lastcharttime = {}
                    lastvalue = {}
                    oldstayid = row['icustay_id']

                if not pd.isna(temp_copy.at[index, col]):
                    lastcharttime[col] = temp_copy.at[index, 'charttime']
                    lastvalue[col] = temp_copy.at[index, col]

                if index > 0:
                    if pd.isna(temp_copy.at[index, col]) and temp_copy.at[index, 'icustay_id'] == oldstayid and (temp_copy.at[index, 'charttime'] - lastcharttime.get(col,0)) <= vitalslab_hold.at[0,col]*3600:
                        temp_copy.at[index, col] = lastvalue.get(col, 0)
                        
    return temp_copy
            

In [19]:
# estimate GCS from RASS - data from Wesley JAMA 2003
reformat.loc[pd.isna(reformat['GCS']) & (reformat['RASS']>=0), 'GCS'] = 15
reformat.loc[pd.isna(reformat['GCS']) & (reformat['RASS']==-1), 'GCS'] = 14
reformat.loc[pd.isna(reformat['GCS']) & (reformat['RASS']==-2), 'GCS'] = 12
reformat.loc[pd.isna(reformat['GCS']) & (reformat['RASS']==-3), 'GCS'] = 11
reformat.loc[pd.isna(reformat['GCS']) & (reformat['RASS']==-4), 'GCS'] = 6
reformat.loc[pd.isna(reformat['GCS']) & (reformat['RASS']==-5), 'GCS'] = 3

# Fi02
reformat.loc[(~pd.isna(reformat['FiO2_100'])) & (pd.isna(reformat['FiO2_1'])), 'FiO2_1'] = reformat['FiO2_100']/100
reformat.loc[(pd.isna(reformat['FiO2_100'])) & (~(pd.isna(reformat['FiO2_1']))), 'FiO2_100'] = reformat['FiO2_1']*100

# ESTIMATE FiO2 /// with use of interface / device (cannula, mask, ventilator....)
reformatsah = SAH(reformat, sample_and_hold) # do SAH first to handle this task

# NO FiO2, YES O2 flow, no interface OR cannula
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)) & (reformatsah['O2flow'] <= 15), 'FiO2_100'] = 70
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)) & (reformatsah['O2flow'] <= 12), 'FiO2_100'] = 62
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)) & (reformatsah['O2flow'] <= 10), 'FiO2_100'] = 55
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)) & (reformatsah['O2flow'] <= 8), 'FiO2_100'] = 50
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)) & (reformatsah['O2flow'] <= 6), 'FiO2_100'] = 44
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)) & (reformatsah['O2flow'] <= 5), 'FiO2_100'] = 40
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)) & (reformatsah['O2flow'] <= 4), 'FiO2_100'] = 36
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)) & (reformatsah['O2flow'] <= 3), 'FiO2_100'] = 32
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)) & (reformatsah['O2flow'] <= 2), 'FiO2_100'] = 28
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)) & (reformatsah['O2flow'] <= 1), 'FiO2_100'] = 24

# NO FiO2, NO O2 flow, no interface OR cannula
reformat.loc[pd.isna(reformatsah['FiO2_100']) & pd.isna(reformatsah['O2flow']) & ((reformatsah['Interface'] == 0) | (reformatsah['Interface'] == 2)), 'FiO2_100'] = 21

# NO FiO2, YES O2 flow, face mask OR.... OR ventilator (assume it's face mask)
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) &\
             ((pd.isna(reformatsah['Interface'])) | (reformatsah['Interface'] == 1) | (reformatsah['Interface'] == 3) | (reformatsah['Interface'] == 4) | (reformatsah['Interface'] == 5) | (reformatsah['Interface'] == 6) | (reformatsah['Interface'] == 9) | (reformatsah['Interface'] == 10)) &\
             (reformatsah['O2flow'] <= 15), 'FiO2_100'] = 75

reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) &\
             ((pd.isna(reformatsah['Interface'])) | (reformatsah['Interface'] == 1) | (reformatsah['Interface'] == 3) | (reformatsah['Interface'] == 4) | (reformatsah['Interface'] == 5) | (reformatsah['Interface'] == 6) | (reformatsah['Interface'] == 9) | (reformatsah['Interface'] == 10)) &\
             (reformatsah['O2flow'] <= 12), 'FiO2_100'] = 69

reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) &\
             ((pd.isna(reformatsah['Interface'])) | (reformatsah['Interface'] == 1) | (reformatsah['Interface'] == 3) | (reformatsah['Interface'] == 4) | (reformatsah['Interface'] == 5) | (reformatsah['Interface'] == 6) | (reformatsah['Interface'] == 9) | (reformatsah['Interface'] == 10)) &\
             (reformatsah['O2flow'] <= 10), 'FiO2_100'] = 66

reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) &\
             ((pd.isna(reformatsah['Interface'])) | (reformatsah['Interface'] == 1) | (reformatsah['Interface'] == 3) | (reformatsah['Interface'] == 4) | (reformatsah['Interface'] == 5) | (reformatsah['Interface'] == 6) | (reformatsah['Interface'] == 9) | (reformatsah['Interface'] == 10)) &\
             (reformatsah['O2flow'] <= 8), 'FiO2_100'] = 58

reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) &\
             ((pd.isna(reformatsah['Interface'])) | (reformatsah['Interface'] == 1) | (reformatsah['Interface'] == 3) | (reformatsah['Interface'] == 4) | (reformatsah['Interface'] == 5) | (reformatsah['Interface'] == 6) | (reformatsah['Interface'] == 9) | (reformatsah['Interface'] == 10)) &\
             (reformatsah['O2flow'] <= 6), 'FiO2_100'] = 40

reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) &\
             ((pd.isna(reformatsah['Interface'])) | (reformatsah['Interface'] == 1) | (reformatsah['Interface'] == 3) | (reformatsah['Interface'] == 4) | (reformatsah['Interface'] == 5) | (reformatsah['Interface'] == 6) | (reformatsah['Interface'] == 9) | (reformatsah['Interface'] == 10)) &\
             (reformatsah['O2flow'] <= 4), 'FiO2_100'] = 36

# NO FiO2, NO O2 flow, face mask OR ....OR ventilator

reformat.loc[pd.isna(reformatsah['FiO2_100']) & pd.isna(reformatsah['O2flow']) &\
             ((pd.isna(reformatsah['Interface'])) | (reformatsah['Interface'] == 1) | (reformatsah['Interface'] == 3) | (reformatsah['Interface'] == 4) | (reformatsah['Interface'] == 5) | (reformatsah['Interface'] == 6) | (reformatsah['Interface'] == 9) | (reformatsah['Interface'] == 10)), 'FiO2_100'] = np.nan

# NO FiO2, YES O2 flow, Non rebreather mask

reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & (reformatsah['Interface'] == 7) & (reformatsah['O2flow'] >= 10) , 'FiO2_100'] = 90
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & (reformatsah['Interface'] == 7) & (reformatsah['O2flow'] >= 15) , 'FiO2_100'] = 100
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & (reformatsah['Interface'] == 7) & (reformatsah['O2flow'] < 10) , 'FiO2_100'] = 80
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & (reformatsah['Interface'] == 7) & (reformatsah['O2flow'] <= 8) , 'FiO2_100'] = 70
reformat.loc[pd.isna(reformatsah['FiO2_100']) & ~pd.isna(reformatsah['O2flow']) & (reformatsah['Interface'] == 7) & (reformatsah['O2flow'] <= 6) , 'FiO2_100'] = 60

# NO FiO2, NO O2 flow, NRM

reformat.loc[pd.isna(reformatsah['FiO2_100']) & pd.isna(reformatsah['O2flow']) & (reformatsah['Interface'] == 7), 'FiO2_100'] = np.nan

# update FiO2 columns again
reformat.loc[(~pd.isna(reformat['FiO2_100'])) & (pd.isna(reformat['FiO2_1'])), 'FiO2_1'] = reformat['FiO2_100']/100
reformat.loc[(pd.isna(reformat['FiO2_100'])) & (~(pd.isna(reformat['FiO2_1']))), 'FiO2_100'] = reformat['FiO2_1']*100

# BP
reformat.loc[~pd.isna(reformat['SysBP']) & ~pd.isna(reformat['MeanBP']) & pd.isna(reformat['DiaBP']), 'DiaBP'] = (3*reformat['MeanBP'] - reformat['SysBP'])/2
reformat.loc[~pd.isna(reformat['SysBP']) & pd.isna(reformat['MeanBP']) & ~pd.isna(reformat['DiaBP']), 'MeanBP'] = (reformat['SysBP'] + 2*reformat['DiaBP'])/3
reformat.loc[pd.isna(reformat['SysBP']) & ~pd.isna(reformat['MeanBP']) & ~pd.isna(reformat['DiaBP']), 'SysBP'] = 3*reformat['MeanBP'] - 2*reformat['DiaBP']

# TEMP
# some values recorded in the wrong column
reformat.loc[(reformat['Temp_F']>25) & (reformat['Temp_F']<45), 'Temp_C'] = reformat['Temp_F']
reformat.loc[(reformat['Temp_F']>25) & (reformat['Temp_F']<45), 'Temp_F'] = np.nan
reformat.loc[reformat['Temp_C']>70, 'Temp_F'] = reformat['Temp_C']
reformat.loc[reformat['Temp_C']>70, 'Temp_C'] = np.nan
reformat.loc[~pd.isna(reformat['Temp_C']) & pd.isna(reformat['Temp_F']), 'Temp_F'] = reformat['Temp_C']*1.8+32
reformat.loc[pd.isna(reformat['Temp_C']) & ~pd.isna(reformat['Temp_F']), 'Temp_C'] = (reformat['Temp_F']-32)/1.8

# Hb/Ht
reformat.loc[~pd.isna(reformat['Hb']) & pd.isna(reformat['Ht']), 'Ht'] = (reformat['Hb']*2.862)+1.216
reformat.loc[pd.isna(reformat['Hb']) & ~pd.isna(reformat['Ht']), 'Hb'] = (reformat['Ht']-1.216)/2.862

# BILI
reformat.loc[~pd.isna(reformat['Total_bili']) & pd.isna(reformat['Direct_bili']), 'Direct_bili'] = (reformat['Total_bili']*0.6934)-0.1752
reformat.loc[pd.isna(reformat['Total_bili']) & ~pd.isna(reformat['Direct_bili']), 'Total_bili'] = (reformat['Direct_bili']+0.1752)/0.6934

In [20]:
# Sample and Hold on raw data
reformat = SAH(reformat, sample_and_hold) 

# Data combination

In [21]:
# WARNING: the time window of interest has been defined above (here -48 -> +24)

timestep = 1 # resolution of timesteps, in hours
icustayidlist = pd.unique(reformat['icustay_id'])
num_of_patients = len(icustayidlist)
reformat2 = pd.DataFrame()

logging.info('Data combination')
for i in range(num_of_patients):
    logging.info('On patient: ' + str(i) + '/' + str(num_of_patients))
    icustayid = icustayidlist[i] # 1 to 100,000 not 200 to 300k
    
    temp = reformat.loc[reformat['icustay_id'] == icustayid]
    first_timestp = temp.loc[temp['timestep'] == 0, 'charttime'].values[0]
        
    # IV Fluid stuff
    inputMV_subset = inputMV.loc[inputMV['icustay_id'] == icustayid + 200000]
    inputCV_subset = inputCV.loc[inputCV['icustay_id'] == icustayid + 200000]
    
    starttime = inputMV_subset['starttime']
    endtime = inputMV_subset['endtime']
    rate = inputMV_subset['normrate']
    
    preadmission = inputpreadm.loc[inputpreadm['icustay_id'] == icustayid + 200000, 'inputpreadm']
    if len(preadmission)>0:
        total_volume = preadmission.sum()
    else:
        total_volume = 0
        
    # compute volume of fluid given before start of record!
    t0 = 0
    t1 = first_timestp
    
    # input from MV (4 ways to compute)
    infusion = (rate*(endtime-starttime)*((endtime <= t1) & (starttime >= t0))/3600 +\
                rate*(endtime - t0)*((starttime <= t0) & (endtime <= t1) & (endtime >= t0))/3600 +\
                rate*(t1 - starttime)*((starttime >= t0) & (endtime >= t1) & (starttime <= t1))/3600 +\
                rate*(t1-t0)*((endtime >= t1) & (starttime <= t0))/3600).sum()
    # all boluses received during this timestep, from inputMV (need to check rate is NaN) and inputCV (simpler):
    bolus = (inputMV_subset.loc[pd.isna(inputMV_subset['rate']) & (inputMV_subset['starttime'] >= t0) & (inputMV_subset['starttime'] <= t1), 'tev'].sum() +\
             inputCV_subset.loc[(inputCV_subset['charttime'] >= t0) & (inputCV_subset['charttime'] <= t1), 'tev'].sum())
    
    total_volume = total_volume + infusion + bolus
    
    # Vasopressors
    vasoMV_subset = vasoMV.loc[vasoMV['icustay_id'] == icustayid + 200000]
    vasoCV_subset = vasoCV.loc[vasoCV['icustay_id'] == icustayid + 200000]
    start_vp = vasoMV_subset['starttime'] #start of vp infusion
    end_vp = vasoMV_subset['endtime']
    rate_vp = vasoMV_subset['rate_std']
    
    # DEMOGRAPHICS / gender, age, elixhauser, re-admit, died in hosp?, died within
    # 48h of out_time (likely in ICU or soon after), died within 90d after admission?
    demog_subset = demog.loc[demog['icustay_id'] == icustayid + 200000]
    qstime_subset = qstime.loc[qstime['icustay_id'] == icustayid]
    if len(demog_subset) > 0:
        demographics = {'gender':demog_subset['gender'].values[0],\
                        'age': demog_subset['age'].values[0],\
                        'elixhauser':demog_subset['elixhauser'].values[0],\
                        're_admission': int(demog_subset['adm_order'].values[0] > 1),\
                        'died_in_hosp': demog_subset['morta_hosp'].values[0],\
                        'died_within_48h_of_out_time': int(abs(demog_subset['dod'].values[0] - demog_subset['outtime'].values[0]) < 24*3600*2),\
                        'mortality_90d': demog_subset['morta_90'].values[0]}
    if len(qstime_subset) > 0:
        demographics.update({'delay_end_of_record_and_discharge_or_death': (qstime_subset['lasttimestp'].values[0] - qstime_subset['firsttimestp'].values[0])/3600})
    
    # URINE OUTPUT
    UO_subset = UO.loc[UO['icustay_id'] == icustayid + 200000]
    preadmission = UOpreadm.loc[UOpreadm['icustay_id'] == icustayid, 'value'] #preadmission UO
    if len(preadmission) > 0:
        UO_total = preadmission.sum()
    else:
        UO_total = 0
        
    UOnow = UO_subset.loc[(UO_subset['charttime'] >= t0) & (UO_subset['charttime'] >= t1), 'value'].sum()
    UO_total = UO_total + UOnow
    
    for j in range(0,80,timestep):
        t0 = 3600*j+ first_timestp
        t1 = 3600*(j + timestep)+ first_timestp
        temp_subset = temp.loc[temp['charttime'].between(t0,t1)]
        if len(temp_subset) > 0:
            data = {'bloc': (j/timestep)+1,\
                    'icustay_id': icustayid,\
                    'charttime': t0}
            data.update(demographics)
            data.update(temp_subset.drop(columns = ['timestep', 'icustay_id', 'charttime']).mean().to_dict())            
            
            
            #VASOPRESSORS
            # for CV: dose at timestamps.
            # for MV: 4 possibles cases, each one needing a different way to compute the dose of VP actually administered:
            #----t0---start----end-----t1----
            #----start---t0----end----t1----
            #-----t0---start---t1---end
            #----start---t0----t1---end----
            
            # MV
            ratemv = vasoMV_subset.loc[(vasoMV_subset['endtime'].between(t0,t1)) |\
                                      ((vasoMV_subset['starttime'] >= t0) & (vasoMV_subset['endtime'] <= t1)) |\
                                      (vasoMV_subset['starttime'].between(t0,t1)) |\
                                      ((vasoMV_subset['starttime'] <= t0) & (vasoMV_subset['endtime'] >= t1)), 'rate_std']
            #CV
            ratecv = vasoCV_subset.loc[vasoCV_subset['charttime'].between(t0,t1), 'rate_std']
            median_rate = pd.concat([ratemv,ratecv]).median()
            max_rate = pd.concat([ratemv,ratecv]).max()
            data.update({'median_dose_vaso': median_rate, 'max_dose_vaso': max_rate})
            
            # MV
            # input from MV (4 ways to compute)
            infusion = (rate*(endtime-starttime)*((endtime <= t1) & (starttime >= t0))/3600 +\
                        rate*(endtime - t0)*((starttime <= t0) & (endtime <= t1) & (endtime >= t0))/3600 +\
                        rate*(t1 - starttime)*((starttime >= t0) & (endtime >= t1) & (starttime <= t1))/3600 +\
                        rate*(t1-t0)*((endtime >= t1) & (starttime <= t0))/3600).sum()
            # all boluses received during this timestep, from inputMV (need to check rate is NaN) and inputCV (simpler):
            bolus = (inputMV_subset.loc[pd.isna(inputMV_subset['rate']) & (inputMV_subset['starttime'] >= t0) & (inputMV_subset['starttime'] <= t1), 'tev'].sum() +\
                     inputCV_subset.loc[(inputCV_subset['charttime'] >= t0) & (inputCV_subset['charttime'] <= t1), 'tev'].sum())

            total_volume = total_volume + infusion + bolus
            
            data.update({'input_total': total_volume, 'input_1hourly': infusion + bolus})
            
            # UO
            UOnow = UO_subset.loc[(UO_subset['charttime'] >= t0) & (UO_subset['charttime'] >= t1), 'value'].sum()
            UO_total = UO_total + UOnow
            
            data.update({'output_total': UO_total, 'output_1hourly': UOnow, 'cumulated_balance': total_volume - UO_total})
            reformat2 = reformat2.append(data, ignore_index=True)
                                                                                                          

If values have less than 75% missing values (over 25% of values present): We keep them

In [22]:
reformat2_filtered = reformat2.copy()
attribute_cols = ['bloc', 'icustay_id', 'charttime', 'gender', 'age', 'elixhauser', 're_admission', 'died_in_hosp', 'died_within_48h_of_out_time', 'mortality_90d', 'delay_end_of_record_and_discharge_or_death', 'extubated', 'mechvent', 'median_dose_vaso', 'max_dose_vaso', 'input_total', 'input_1hourly', 'output_total', 'output_1hourly', 'cumulated_balance']
for col in reformat2_filtered:
    if col not in attribute_cols:
        miss = reformat2_filtered[col].isnull().sum() / len(reformat2_filtered)
        if miss > 0.75:
            reformat2_filtered.drop(columns=col, inplace = True)

reformat2_filtered.sort_values(by=['icustay_id', 'bloc'], inplace=True)

# Handling of missing values

In [23]:
reformat3 = reformat2_filtered.copy()
for col in reformat2:
    if col not in attribute_cols:
        miss = reformat2[col].isnull().sum()/len(reformat2)
        if miss > 0 and miss <= 0.05:
            reformat3[col] = reformat3[col].interpolate(method='linear').fillna(0)
            

In [24]:
len(reformat2_filtered.columns)

70

In [25]:
# KNN Imputation - Done on chunks of 10K records
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=1)

value_cols = set(reformat3.columns) - set(attribute_cols)
ref_values = reformat3[value_cols].copy()
ind = 0
while(ind < len(ref_values)):
    ref_values.loc[ind:ind+10000, value_cols] = imputer.fit_transform(ref_values.loc[ind:ind+10000, value_cols])
    ind = ind + 10000

reformat4 = pd.concat(axis = 1, objs = [reformat3[attribute_cols], ref_values])   


# Compute some derived variables: P/F, Shock Index, SOFA, SIRS...

In [26]:
# Correct gender
reformat4['gender'] = reformat4['gender']-1

In [27]:
# Correct Age > 200 yo
reformat4.loc[reformat4['age']>150*365.25, 'age'] = 91.4*365.25

# Fix Mechvent
reformat4['mechvent'] = reformat4['mechvent'].fillna(0)
reformat4.loc[reformat4['mechvent']>0, 'mechvent'] = 1

# Fix Elixhauser missing values
reformat4['elixhauser'] = reformat4['elixhauser'].fillna(reformat4['elixhauser'].median())

# vasopressors / no NAN
if 'median_dose_vaso' in reformat4:
    reformat4['median_dose_vaso'] = reformat4['median_dose_vaso'].fillna(0)
if 'max_dose_vaso' in reformat4:
    reformat4['max_dose_vaso'] = reformat4['max_dose_vaso'].fillna(0)
    
# re-compute P/F with no missing values
reformat4['PaO2_FiO2'] = reformat4['paO2']/reformat4['FiO2_1']

# re-compute SHOCK INDEX without NaN and Inf
reformat4['Shock_Index'] = reformat4['HR']/reformat4['SysBP']
reformat4.replace([np.inf, -np.inf], np.nan, inplace=True)
reformat4['Shock_Index'] = reformat4['Shock_Index'].fillna(reformat4['Shock_Index'].mean())


In [28]:
# SOFA - at each timepoint
# need (in this order):  P/F  MV  PLT  TOT_BILI  MAP  NORAD(max)  GCS  CR  UO
def score_pf(row):
    if row['PaO2_FiO2'] < 100:
        return 4
    if row['PaO2_FiO2'] < 200:
        return 3
    if row['PaO2_FiO2'] < 300:
        return 2
    if row['PaO2_FiO2'] < 400:
        return 1
    else:
        return 0

def score_plt(row):
    if row['Platelets_count'] < 20:
        return 4
    if row['Platelets_count'] < 50:
        return 3
    if row['Platelets_count'] < 100:
        return 2
    if row['Platelets_count'] < 150:
        return 1
    else:
        return 0
    
def score_tot_bili(row):
    if row['Total_bili'] > 12:
        return 4
    if row['Total_bili'] >= 6:
        return 3
    if row['Total_bili'] >= 2:
        return 2
    if row['Total_bili'] >= 1.2:
        return 1
    else:
        return 0

def score_bp_vp(row):
    if row['max_dose_vaso'] > 0.1:
        return 4
    if row['max_dose_vaso'] > 0:
        return 3
    if row['MeanBP'] < 65:
        return 2
    if row['MeanBP'] < 70:
        return 1
    else:
        return 0
    
def score_gcs(row):
    if row['GCS'] <= 5:
        return 4
    if row['GCS'] <= 9:
        return 3
    if row['GCS'] <= 12:
        return 2
    if row['GCS'] <= 14:
        return 1
    else:
        return 0
    
def score_cr_uo(row):
    if row['output_1hourly'] < 34 or row['Creatinine'] > 5:
        return 4
    if row['output_1hourly'] < 84 or (row['Creatinine'] >= 3.5 and row['Creatinine'] < 5):
        return 3
    if row['Creatinine'] >= 2:
        return 2
    if row['Creatinine'] >= 1.2:
        return 1
    else:
        return 0    

In [29]:
def sofa_score(row):
    return score_pf(row) + score_plt(row) + score_tot_bili(row) + score_bp_vp(row) + score_gcs(row) + score_cr_uo(row)

In [30]:
for index, row in reformat4.iterrows():
    sofa = sofa_score(row)
    reformat4.at[index, 'SOFA'] = sofa

In [31]:
# SIRS - at each timepoint
reformat4['SIRS'] = ((reformat4['Temp_C']>=38) | (reformat4['Temp_C']<=36)).astype('int32') +\
(reformat4['HR']>90).astype('int32') +\
((reformat4['RR']>=20)|(reformat4['paCO2']<=32)).astype('int32') +\
((reformat4['WBC_count']>=12)|(reformat4['WBC_count']<4)).astype('int32') 


In [32]:
# more IO corrections
reformat4.loc[reformat4['input_total']<0, 'input_total'] = 0
reformat4.loc[reformat4['input_1hourly']<0, 'input_1hourly'] = 0

# Exclusion of some patients

In [33]:
patients = reformat4['icustay_id'].unique()
logging.info('Excluding some patients')
logging.info('Number of patients before: ' + str(len(patients)))

# check for patients with extreme UO = outliers = to be deleted (>40 litres of UO per 4h!!)
outliers = set(reformat4.loc[reformat4['output_1hourly'] > 12000, 'icustay_id'])

# some have bili = 999999
outliers = outliers.union(reformat4.loc[reformat4['Total_bili'] > 10000, 'icustay_id'])

# check for patients with extreme INTAKE = outliers = to be deleted (>10 litres of intake per 4h!!)
outliers = outliers.union(reformat4.loc[reformat4['input_1hourly'] > 10000, 'icustay_id'])

#### exclude early deaths from possible withdrawals ####
for id_ in patients:
    if reformat4.loc[reformat4['icustay_id'] == id_, 'mortality_90d'].max() == 1:
        total_blocs = reformat4.loc[reformat4['icustay_id'] == id_, 'bloc'].max()
        max_vaso = reformat4.loc[reformat4['icustay_id']==id_, 'max_dose_vaso'].max()
        max_sofa = reformat4.loc[reformat4['icustay_id']==id_, 'SOFA'].max()
        last_row = reformat4.loc[(reformat4['icustay_id'] == id_) & (reformat4['bloc'] == total_blocs)]
        if max_vaso > 0.3 and ((last_row['max_dose_vaso']==0) & (last_row['SOFA'] >= max_sofa/2)).sum() > 0 and total_blocs<20:
            outliers = outliers.union([id_])

# exclude patients who died in ICU during data collection period
outliers = outliers.union(reformat4.loc[(reformat4['bloc'] == 0) & (reformat4['died_within_48h_of_out_time']==1) & (reformat4['delay_end_of_record_and_discharge_or_death'] < 24), 'icustay_id'])

for index, row in reformat4.iterrows():
    if row['icustay_id'] in outliers:
        reformat4.drop(labels=index, inplace=True)
        
patients = reformat4['icustay_id'].unique()
logging.info('Number of patients after: ' + str(len(patients)))

Number of patients before:  21363
Number of patients after:  17145


In [34]:
columns_to_be_kept = ['bloc','icustay_id','charttime','gender','age','elixhauser','re_admission', 'died_in_hosp', 'died_within_48h_of_out_time','mortality_90d','delay_end_of_record_and_discharge_or_death','SOFA','SIRS',\
    'Weight_kg','GCS','HR','SysBP','MeanBP','DiaBP','RR','SpO2','Temp_C','FiO2_1','Potassium','Sodium','Chloride','Glucose',\
    'BUN','Creatinine','Magnesium','Calcium','Ionised_Ca','CO2_mEqL','SGOT','SGPT','Total_bili','Albumin','Hb','WBC_count','Platelets_count','PTT','PT','INR',\
    'Arterial_pH','paO2','paCO2','Arterial_BE','HCO3','Arterial_lactate','mechvent','Shock_Index','PaO2_FiO2',\
    'median_dose_vaso','max_dose_vaso','input_total','input_1hourly','output_total','output_1hourly','cumulated_balance']


MIMICtable = reformat4[columns_to_be_kept]
MIMICtable.head()

,bloc,icustay_id,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,mechvent,Shock_Index,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_4hourly,output_total,output_4hourly,cumulated_balance
0,1.0,3.0,7.245400e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.653782,222.499997,0.0,0.0,6297.0,30.0,9090.0,4305.0,-2793.0
1,2.0,3.0,7.245414e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.710438,207.499997,0.0,0.0,6347.0,50.0,13060.0,3970.0,-6713.0
2,3.0,3.0,7.245428e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.748397,207.499997,0.0,0.0,6397.0,50.0,16260.0,3200.0,-9863.0
3,4.0,3.0,7.245443e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.615226,207.499997,0.0,0.0,6447.0,50.0,18910.0,2650.0,-12463.0
4,5.0,3.0,7.245457e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.749047,165.573772,0.0,0.0,6477.0,30.0,21180.0,2270.0,-14703.0


In [35]:
MIMICtable.to_csv(exportdir + '/MIMICtable-1hourly.csv', index=False)

# Create Sepsis Cohort
Create array with 1 row per icu admission  
Keep only patients with flagged sepsis (max sofa during time period of interest >= 2)  
We assume baseline SOFA of zero (like other publications)  

In [36]:
sepsis = pd.DataFrame()
for icustayid in range(1, 100001):
    reformat4_subset = reformat4.loc[reformat4['icustay_id'] == icustayid]
    if len(reformat4_subset) > 0:
        max_sofa = reformat4_subset['SOFA'].max()
        max_sirs = reformat4_subset['SIRS'].max()
        morta_90 = reformat4_subset['mortality_90d'].values[0]
        onsettime = onset.loc[onset['icustay_id'] == icustayid, 'onsettime'].max()
        sepsis = sepsis.append({'icustayid': icustayid,\
                       'morta_90d': morta_90,\
                       'max_sofa': max_sofa,\
                       'max_sirs': max_sirs,\
                       'sepsis_time': onsettime}, ignore_index=True)


In [37]:
sepsis.head()

,icustayid,max_sirs,max_sofa,morta_90d,sepsis_time
0,3.0,3.0,12.0,1.0,7.245590e+09
1,11.0,1.0,10.0,0.0,6.898284e+09
2,12.0,3.0,8.0,0.0,5.805734e+09
3,14.0,2.0,9.0,1.0,4.264272e+09
4,38.0,1.0,9.0,0.0,5.484931e+09


In [38]:
sepsis.to_csv(exportdir + '/sepsis-1hourly.csv', index = False)